## Direct API Call Using OpenAI

You can make direct API calls using either:

1. **OpenAI-hosted models**, or  
2. **Local models running on your machine** (demonstrated in this notebook).

This notebook primarily uses a **local model**, but the same structure applies to OpenAI models with minor changes.


In [60]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

##### Note:
    If you are have or using openai api key,
    1. Remove the base_url part.
    2. Give the name of the llm models provided by openai instead.

In [ ]:
from openai import OpenAI
client = OpenAI(api_key= os.environ["OPENAI_API_KEY"], base_url= os.environ["BASE_URL"])
llm_model = "qwen/qwen3-8b"

In [89]:
prompt = "Write a story of a brave knight and a dragon in 100 words."

In [62]:
def get_response(prompt,llm_model):
    messages = [{"role":"user", "content":prompt}]
    response = client.chat.completions.create(
        model=llm_model,
        messages=messages,
        temperature=0.0
    )
    return response.choices[0].message.content

In [63]:
get_response(prompt,llm_model)

"<think>\nOkay, the user wants a 100-word story about a brave knight and a dragon. Let me start by setting up the scene. Maybe a medieval kingdom under threat from a dragon. The knight needs to be brave, so I'll give him a noble name like Sir Cedric. The dragon should be fearsome but not too evil, maybe with some unique traits like glowing scales or a specific color. I need to keep the word count tight.\n\nI should introduce conflict: the knight is tasked with slaying the dragon. Maybe include some dialogue or internal struggle. Keep descriptions concise. Avoid unnecessary details. Make sure the story has a clear beginning, middle, and end. Check for flow and ensure it's engaging but not too long. Let me draft a quick version first.\n</think>\n\n**The Dragon’s Trial**  \n\nSir Cedric, a knight of honor, rode into the misty mountains where the ancient dragon, Ignarion, dwelled. Its scales glowed crimson, its breath hot as molten iron. For years, it had guarded the realm, but now, it sou

### Important Note for OpenAI API Users

If you are using an **OpenAI API key**:

1. Remove any `base_url` parameter intended for local model inference.
2. Use one of the official OpenAI model names (e.g., `gpt-4.1-mini`, `gpt-4.1`, `gpt-4.1-preview`, etc.).

In [90]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="qwen/qwen3-8b", api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("BASE_URL"))
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020217154B00>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020217109D50>, root_client=<openai.OpenAI object at 0x0000020217145A90>, root_async_client=<openai.AsyncOpenAI object at 0x0000020217109A50>, model_name='qwen/qwen3-8b', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://172.30.128.1:3000/v1')

## Using a Prompt Template

Prompt templates help you structure instructions in a reusable and consistent format.  
They allow you to insert variables such as user input, style, or context dynamically.

### 1. Basic Prompt Template

A simple prompt template contains static text without any variables.  
It is useful when you want to reuse the same instruction multiple times.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = "Write a story of a brave knight and a dragon in 100 words."

prompt_template = ChatPromptTemplate.from_template(prompt)
format_instructions = prompt_template.format_messages()

chat.invoke(format_instructions)

AIMessage(content='<think>\nOkay, the user wants me to write a story about a brave knight and a dragon in 100 words. Let me start by breaking down the requirements.\n\nFirst, I need to include both characters: a brave knight and a dragon. The story should be concise since it\'s limited to 100 words. Maybe set up a conflict between them? Like a battle or some kind of challenge where they have to work together or maybe compete against each other?\n\nI should make sure the narrative is engaging but not too long. Maybe start with introducing the knight, perhaps in a medieval setting since dragons are common in fantasy stories. The dragon could be a guardian spirit or an ancient creature. They might have a rivalry or mutual goal.\n\nI need to check if there\'s a plot structure: introduction, rising action, climax, resolution. Let me think of possible scenarios. Maybe the knight is on a quest and encounters the dragon, which can either help or hinder them. The story should be self-contained,

### 2. Advanced Prompt Template

Advanced templates support **placeholders (variables)** such as `{name}`, `{style}`, or `{text}`.  
These enable dynamic, flexible message construction based on input provided at runtime.

In [94]:
from langchain_core.prompts import ChatPromptTemplate

template_string = """Translate the text 
that is delimited by triple backticks 
into a style that is {style}. 
text: ```{text}```
"""

# Create a reusable prompt template
prompt_template = ChatPromptTemplate.from_template(template_string)

In [95]:
service_reply = """Hey there customer, 
the warranty does not cover 
cleaning expenses for your kitchen 
because it's your fault that 
you misused your blender 
by forgetting to put the lid on. Tough luck!"""

service_style_pirate = "a polite tone that speaks in English Pirate"

service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

service_response = chat.invoke(service_messages)
print(service_response.content)
# chat(format_instructions)

<think>
Okay, so I need to translate this text from a specific format into a more polite and Pirate-style English tone. Let me first read through the original text carefully.

The original text is enclosed within triple backticks, which probably indicates that it's a code block or some kind of formatted input. The content inside says something like "Hey there customer," followed by a message about warranty not covering cleaning expenses for the kitchen because of misuse of the blender without putting the lid on. It ends with "Tough luck!"

Now, the user wants this translated into a style that's polite but still has a Pirate tone. So I need to make sure that the translation is respectful and friendly rather than being harsh or overly direct. The Pirate style might mean using more colloquial language, maybe some slang, while keeping it in English.

Let me start by breaking down each part of the original text:

1. "Hey there customer," – This is an address to the recipient. In a polite to

## Parsers

Parsers are essential when working with LLMs because model outputs are often free-form text.  
Parsers help convert this text into structured formats.

### 1. Raw Parsing

This returns the output **as plain unstructured text**.  
Useful for simple queries where no structured format is required.

In [104]:
context= """My name is Ajith Narayanan. I went to Delhi on 24-11-2025. I had a great 
meeting with the captain of the Indian men's cricket
team, Virat Kohli. He even gave me an autograph.
"""
prompt = """Based on the given context, create a JSON file in the given format.
JSON format:
Name:
Date:
Location:
Summary:

Context: {context}"""

prompt_template = ChatPromptTemplate.from_template(prompt)
format_instructions = prompt_template.format_messages(context= context)
result = chat.invoke(format_instructions)
print(result.content)

<think>
Okay, let me try to figure this out. The user wants a JSON file based on the given context. Let me look at the example they provided.

The context says: "My name is Ajith Narayanan. I went to Delhi on 24-11-2025. I had a great meeting with the captain of the Indian men's cricket team, Virat Kohli. He even gave me an autograph."

So first, I need to extract the key pieces of information here. The name is Ajith Narayanan. The date mentioned is 24-11-2025, which I should format as a date in JSON. The location is Delhi. The summary mentions a meeting with Virat Kohli, who is the captain of the Indian men's cricket team, and he gave an autograph.

Now, putting this into the specified JSON structure. Let me check if all the required fields are covered: Name, Date, Location, Summary. 

For the date, 24-11-2025 seems like a day in November 2025, so I should format it as "2025-11-24" or maybe just keep it as-is? The example might want it in the standard date format. Let me check if ther

### 2. Structured Output (JSON, List, Dict, etc.)

For structured output, you can use:

1. **LangChain Output Parsers**, which enforce JSON or object-based responses.
2. **Pydantic Models**, which provide strict validation and type safety.

Using structured parsing ensures that LLM responses are clean, validated, and ready for downstream processing.

"""

In [110]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from datetime import datetime
from typing import Literal, List

class Details(BaseModel):
    Name : str = Field(description="Full name of the person involved.")
    Date : datetime = Field(description= "Date of the event in ISO format (YYYY-MM-DD).")
    Location: Literal["Mumbai", "Delhi", "Chennai"]  = Field(description="Location where the event happened.")
    Summary: List[str] = Field(description= "A list of summarized statements or key highlights.")
    
parser = JsonOutputParser(pydantic_object = Details)
format_instructions = parser.get_format_instructions()

context= """My name is Ajith Narayanan. I went to Delhi on 24-11-2025. I had a great 
meeting with the captain of the Indian men's cricket
team, Virat Kohli. He even gave me an autograph.
"""
prompt = """Based on the given context, create a JSON file in the given format.
JSON format:
Name:
Date:
Location:
Summary:

Context: {context}"""

prompt_template = ChatPromptTemplate.from_template(prompt)
format_messages = prompt_template.format_messages(
    context = context,
    format_instructions = format_instructions
    )
result = chat.invoke(format_messages)
print(result.content)

<think>
Okay, let's see what the user is asking for here. They want me to create a JSON file based on the given context and specific format. The context provided mentions Ajith Narayanan going to Delhi on 24-11-2025 and meeting with Virat Kohli, who gave him an autograph.

First, I need to parse the information. The user specified that the JSON should have Name, Date, Location, and Summary fields. So, the structure of the JSON will be straightforward. Let me make sure all the required keys are included: "Name", "Date", "Location", "Summary".

Now, for each key:

- **Name**: The context says, "My name is Ajith Narayanan." So the Name should be "Ajith Narayanan".
  
- **Date**: It states, "I went to Delhi on 24-11-2025." That's a date format. However, 24-11-2025 might need formatting as a date in YYYY-MM-DD or just as a string. Since the user hasn't specified, but it's likely that they want the actual date, I'll use "2025-11-24" to match the format.
  
- **Location**: The context says, "

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Create Chat Model
chat = ChatOpenAI(
    model="qwen/qwen3-8b",
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY")
)

# Prompt Template
template_string = """
Translate the text 
that is delimited by triple backticks 
into a style that is {style}. 

text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

service_reply = """Hey there customer, 
the warranty does not cover 
cleaning expenses for your kitchen 
because it's your fault that 
you misused your blender 
by forgetting to put the lid on. Tough luck!"""

service_style_pirate = "a polite tone that speaks in English Pirate"

# Format the messages
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

# Call the model
service_response = chat.invoke(service_messages)
print(service_response.content)


<think>
Okay, I need to translate this text into a polite tone in English Pirate style. Let me start by understanding the original message.

The original text is a customer service response explaining that the warranty doesn't cover cleaning costs because the customer didn't use the blender correctly by forgetting to put the lid on. It's pretty straightforward but has a somewhat harsh tone. 

Now, I need to convert this into a polite Pirate style. The key here is to maintain the message's intent while making it sound more friendly and approachable. Pirates often use a conversational and slightly informal tone, so I should aim for that.

First, I'll start by addressing the customer warmly, maybe with something like "Ahoy there, matey!" to set a friendly vibe. Then, I need to explain the warranty exclusion politely. Instead of saying it doesn't cover cleaning expenses because of their mistake, perhaps rephrase it as "the warranty doesn't include cleaning costs due to an oversight in usin